In [1]:
import numpy as np
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import plotly.graph_objects as go
from src.data.Segment_Slicer import SegmentSlicer 
ss = SegmentSlicer()
from scipy.signal import savgol_filter

In [2]:
raw_parquet = pd.read_parquet('../data/raw/reunion_segments.parquet')
df_parquet = pd.DataFrame(raw_parquet)

In [3]:
ss.cut_segment(df_parquet['altitude_profile'].iloc[0], df_parquet['distance_profile'].iloc[0], df_parquet['coordinates'].iloc[0])

[{'type': 'downhill',
  'category': 'Uncategorized',
  'start_distance': np.float64(0.0),
  'end_distance': np.float64(96.0),
  'distance': np.float64(96.0),
  'start_altitude': np.float64(67.2),
  'end_altitude': np.float64(63.6),
  'elevation_gain': 0,
  'elevation_loss': np.float64(3.6000000000000014),
  'elevation_change': np.float64(-3.6000000000000014),
  'grade': np.float64(-2.3396626569104186),
  'max_grade': np.float64(2.0112995145784027),
  'min_grade': np.float64(-5.043683747123649),
  'grade_variance': np.float64(6.042728849353839),
  'start_idx': 0,
  'end_idx': np.int64(9)},
 {'type': 'climb',
  'category': 'Cat 4',
  'start_distance': np.float64(96.0),
  'end_distance': np.float64(2088.5),
  'distance': np.float64(1992.5),
  'start_altitude': np.float64(63.6),
  'end_altitude': np.float64(163.8),
  'elevation_gain': np.float64(99.80000000000001),
  'elevation_loss': 0,
  'elevation_change': np.float64(99.80000000000001),
  'grade': np.float64(5.008782936010038),
  'max_g

In [8]:
def plot_elevation_profile(altitude_profile, distance_profile, coordinates, smooth_window=20):
    segments = ss.cut_segment(altitude_profile, distance_profile, coordinates)
    fig = go.Figure()

    # Smooth the altitude profile if smooth_window is greater than 0
    if smooth_window > 0:
        polyorder = 2 if smooth_window <= 3 else 3
        altitude_profile_smoothed = savgol_filter(altitude_profile, smooth_window, polyorder)
    else:
        altitude_profile_smoothed = altitude_profile

    # Add thick black line for the actual elevation profile with custom hover text
    fig.add_trace(go.Scatter(
        x=distance_profile,
        y=altitude_profile_smoothed,
        mode='lines',
        line=dict(width=2, color='black'),
        name='Elevation Profile',
        hoverinfo='none',  # Disable default hover info
        hovertemplate='<br>'.join([
            '%{customdata[0]}',
            '%{customdata[1]}'
        ])+'<extra></extra>',
        customdata=np.stack(([f"📏 {x:.0f}m" for x in distance_profile], [f"🏔️ {y:.0f}m" for y in altitude_profile_smoothed]), axis=-1)
    ))

    min_alt = min(altitude_profile_smoothed)
    baseline = min_alt * 0.9

    # Add colored areas for each segment type below the black line
    for seg in segments:
        if seg['type'] == 'climb':
            color = 'red'
        elif seg['type'] == 'uphill':
            color = 'orange'
        elif seg['type'] == 'flat':
            color = 'blue'
        elif seg['type'] == 'downhill':
            color = 'grey'
        elif seg['type'] == 'descent':
            color = 'green'
        else:
            color = 'blue'

        # Find indices for the segment in the distance_profile
        start_idx = np.searchsorted(distance_profile, seg['start_distance'], side='left')
        end_idx = np.searchsorted(distance_profile, seg['end_distance'], side='right')

        # Create a polygon shape for the filled area
        fig.add_shape(
            type='path',
            path=f'M {distance_profile[start_idx]} {altitude_profile_smoothed[start_idx]} ' +
                 ' '.join([f'L {distance_profile[i]} {altitude_profile_smoothed[i]}' for i in range(start_idx + 1, end_idx)]) +
                 f' L {distance_profile[end_idx-1]} {baseline} ' +
                 ' '.join([f'L {distance_profile[i]} {baseline}' for i in range(end_idx - 2, start_idx - 1, -1)]) +
                 ' Z',
            fillcolor=color,
            opacity=0.5,
            line=dict(width=0)
        )

        # Add annotations for each segment type
        x_mid = (seg['start_distance'] + seg['end_distance']) / 2
        y_mid = (altitude_profile_smoothed[start_idx] + altitude_profile_smoothed[end_idx-1]) / 2
        if seg['type'] == 'climb':
            annotation_text = f"📏 {seg['distance']:.0f}m <br>🏔️ {seg['elevation_change']:.0f}m <br>📈 {seg['grade']:.0f}%"
        elif seg['type'] == 'uphill':
            annotation_text = f"📏 {seg['distance']:.0f}m <br>🏔️ {seg['elevation_change']:.0f}m"
        elif seg['type'] == 'flat':
            annotation_text = f"📏 {seg['distance']:.0f}m"
        elif seg['type'] == 'downhill':
            annotation_text = f"📏 {seg['distance']:.0f}m <br>🏔️ {abs(seg['elevation_change']):.0f}m"
        elif seg['type'] == 'descent':
            annotation_text = f"📏 {seg['distance']:.0f}m <br>🏔️ {abs(seg['elevation_change']):.0f}m <br>📉 {abs(seg['grade']):.0f}%"

        fig.add_annotation(
            x=x_mid,
            y=y_mid + (max(altitude_profile_smoothed) - min(altitude_profile_smoothed)) * 0.1,
            text=annotation_text,
            showarrow=False,
            font=dict(color='black', size=10),
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='black',
            borderwidth=1,
            borderpad=3
        )

    # Update layout
    fig.update_layout(
        title='Trail Elevation Profile with Segment Types',
        xaxis_title='Distance (m)',
        yaxis_title='Altitude (m)',
        yaxis=dict(
            range=[min_alt * 0.9, max(altitude_profile_smoothed) * 1.1],
            showgrid=False
        ),
        xaxis=dict(showgrid=False),
        showlegend=False,
        hovermode='x',
        margin=dict(t=40),
        height=600
    )
    fig.show()

In [9]:
plot_elevation_profile(df_parquet['altitude_profile'].iloc[0], df_parquet['distance_profile'].iloc[0], df_parquet['coordinates'].iloc[0])